In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [ ]:
pos_path = ''
partition_path ''
test_users_path = ''
root_path = ''
output_path = ''

In [ ]:
df_pos = pd.read_csv(pos_path)

In [ ]:
df_neg = []
def get_path(i):
    return f'{root_path}partition-{i}_test.csv'

In [ ]:
for i in range(8):
    df_neg.append(pd.read_csv(get_path(i)))

In [ ]:
df_neg = pd.concat(df_neg)

In [ ]:
test_users = np.load(test_users_path)

In [ ]:
df_pos = df_pos[df_pos['author_id'].isin(test_users)]

In [ ]:
df_pos['hash'] = df_pos[['author_id', 'max_timestamp']].apply(lambda x: hash(tuple(x)), axis = 1)

In [ ]:
df_neg['hash'] = df_neg[['author_id', 'max_timestamp']].apply(lambda x: hash(tuple(x)), axis = 1)

In [ ]:
df_neg = df_neg[df_neg['author_id'].isin(test_users)]

In [ ]:
fix = df_neg.groupby('hash')['article_id'].nunique()

In [ ]:
fix = fix.reset_index()
usable_hashs = fix[fix['article_id'] > 50]['hash']

In [ ]:
df_neg = df_neg[df_neg['hash'].isin(usable_hashs)]

In [ ]:
df_neg = df_neg.drop_duplicates(subset=['hash', 'article_id'])

In [ ]:
df_pos = df_pos[df_pos['hash'].isin(usable_hashs)]

In [ ]:
new_df_pos = df_pos.sample(frac=0.4, random_state=1).reset_index()

In [ ]:
test_dataset = pd.DataFrame(columns=['author_id', 'article_id', 'hash', 'comment_ids'])
progress = 0
for index, row in new_df_pos.iterrows():
    if index % 300 == 0:
        print(index / len(new_df_pos))
    h = row.hash
    row['y'] = 1
    neg_samples = df_neg[df_neg['hash'] == h].head(50)
    neg_samples['y'] = 0
    test_dataset = test_dataset.append(row[['author_id', 'article_id', 'hash', 'comment_ids']], ignore_index=True)
    test_dataset = test_dataset.append(neg_samples[['author_id', 'article_id', 'hash', 'comment_ids']], ignore_index=True)

In [ ]:
print('Number of unique users', new_df_pos['author_id'].nunique())

In [ ]:
len(new_df_pos)

In [ ]:
test_dataset.to_csv(output_path, index=False)

In [ ]:
len(test_dataset)